In [1]:
#imports
import boto3
import pyodbc
import pandas as pd
import numpy as np
import redshift_connector
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import os
import json
from functools import reduce
from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
with open('aws.json') as json_file:
    cr = json.load(json_file)

s3creds = cr['data_sources']['s3']['creds']
redcreds = cr['data_sources']['redshift']['creds']

In [3]:
KEYPATH = "gcp.json"
CREDS = service_account.Credentials.from_service_account_file(KEYPATH)
client = bigquery.Client(credentials=CREDS, project=CREDS.project_id)


q1 = """
SELECT * FROM training_dataset.tournament
"""

view1 = client.query(q1)

tournament = view1.result().to_dataframe()

In [4]:
q2 = """
SELECT * FROM training_dataset.tournament
JOIN training_dataset.tournament_winner ON tournament.row_order = tournament_winner.row_order
WHERE pl1_flag in ('CAN','EGY','FRA')
"""

df = client.query(q2).result().to_dataframe()


In [5]:
#s3 connect 
s3 = boto3.resource(
    service_name='s3',
    region_name=s3creds['region_name'],
    aws_access_key_id=s3creds['aws_access_key_id'],
    aws_secret_access_key=s3creds['aws_secret_access_key']
)

tw_name = 'Tournament_Won.csv'
tl_name = 'Tournament_Lost.csv'

#s3 get data
bucket_name = s3creds['bucket']
tw_file_name = s3creds['folder_path']+tw_name
tl_file_name = s3creds['folder_path']+tl_name
bucket = s3.Bucket(bucket_name)
tw_obj = bucket.Object(key=tw_file_name)
tl_obj = bucket.Object(key=tl_file_name)
tw_response = tw_obj.get()
tl_response = tl_obj.get()
twTable = pd.read_csv(tw_response['Body'],header=3)
tlTable = pd.read_csv(tl_response['Body'],header=3)

In [6]:
RH = df.query('pl1_hand == "Right-Handed" ')

In [7]:
RHWinner = RH.merge(twTable, on='row_order')

In [8]:
RHWinner50 = RHWinner.query('WPts > 50')

In [9]:
RHWinner50Loser = RHWinner50.merge(tlTable,on='row_order')

In [10]:
RHWinner50Loser10 = RHWinner50Loser.query('AvgL < 10')

In [11]:
tdL = date.today() + relativedelta(months=-74)
tdH = date.today() + relativedelta(months=-50)
dateAfter = pd.to_datetime(tdL)
dateBefore = pd.to_datetime(tdH)

In [18]:
date1 = RHWinner50Loser10[pd.to_datetime(RHWinner50Loser10['tournament_date'])>dateAfter]

In [20]:
date2 = date1[pd.to_datetime(date1['tournament_date'])<dateBefore]

In [24]:
stdcol = ['row_order','ATP_x','tournament_date','Winner','pl1_hand','pl1_flag','AvgL']

In [26]:
uploadData = date2[stdcol]

In [27]:
pandas_gbq.to_gbq(uploadData, destination_table = 'training_dataset.DA-15', project_id=CREDS.project_id, credentials = CREDS,if_exists='replace')